In [2]:
# %load tweepy_download.py
#!/usr/bin/env python3
"""
Created on Sun Apr 19 11:37:06 2020

@author: patrickschulze
"""

import pandas as pd
import pickle
import tweepy_helpers as th

# import Bundestag data
with open('abg_df.pickle', 'rb') as handle:
    bt_data = pickle.load(handle)
    
# select name and username for each member and store in table called twitter_account
names = bt_data['Name']
twitter_usernames = bt_data['Twitter']
names.rename("name", inplace = True)
twitter_usernames.rename("username", inplace = True)
twitter_account = pd.concat([names, twitter_usernames], axis = 1)

# drop usernames that are nan or empty (i.e. parliamentarians with no account)
usr_nan = twitter_account.username.isna()
usr_empty = twitter_account.username == ''
twitter_account = twitter_account[~(usr_nan | usr_empty)]
twitter_account.reset_index(drop = True, inplace = True)

In [3]:
# load in tweepy_df 
with open('tweepy_df.pickle', 'rb') as handle:
    tweepy_df = pickle.load(handle)

In [10]:
missing_usernames = tweepy_df.loc[tweepy_df.available == False][['name', 'username']]

In [21]:
# download most recent tweets using tweepy (at most 3200 tweets per user) for initially missing accounts
for username in missing_usernames['username']:
    tweepy_df = pd.concat([tweepy_df, th.download_tweets_tweepy(username)])

data for user Manfredbehrens cannot be downloaded
data for user EspendillerM cannot be downloaded
data for user FechnerJohannes cannot be downloaded
data for user UweFe cannot be downloaded
data for user friedhoff_afd cannot be downloaded
data for user manfred_grund cannot be downloaded
data for user FGuentzler cannot be downloaded
data for user ChristianHaase6 cannot be downloaded
data for user frankheinrich cannot be downloaded
data for user gabihillerohm cannot be downloaded
data for user Karsten_Hilse cannot be downloaded
data for user fjunge cannot be downloaded
data for user lier_e cannot be downloaded
data for user AchimKessler cannot be downloaded
data for user koobmar cannot be downloaded
data for user UlrichLange cannot be downloaded
data for user buerger2016 cannot be downloaded
data for user thlutze cannot be downloaded
data for user AMattfeldt cannot be downloaded
data for user MdBMonstadt cannot be downloaded
data for user WilfriedOellers cannot be downloaded
data for use

In [23]:
# add column 'name'
tweepy_df = twitter_account.merge(tweepy_df, on = 'username')
tweepy_df = tweepy_df.drop(columns = ['name_y'])
tweepy_df.rename(columns={'name_x':'name'}, inplace=True)

In [33]:
# save data
with open('tweepy_df.pickle', 'wb') as handle:
    pickle.dump(tweepy_df, handle, protocol=pickle.HIGHEST_PROTOCOL)